In [1]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from datasets import load_dataset

In [2]:
model_name = "alirezamsh/small100"

In [3]:
#device management 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device) # (, non_blocking=True)

device = get_default_device()
print(device)

cuda


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    tokenizer.tgt_lang = "pt"
    inputs =  example['en_US']
    targets = example['pt_PT']

    return tokenizer(
        inputs, 
        text_target=targets, 
        truncation=True, max_length=128
    )

class Dataset():
    def __init__(self):
        train_dataset = load_dataset("Amani27/massive_translation_dataset", split="train")
        valid_dataset = load_dataset("Amani27/massive_translation_dataset", split="validation")
        test_dataset  = load_dataset("Amani27/massive_translation_dataset", split="test")

        self.tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
        self.tokenized_valid_dataset = valid_dataset.map(tokenize_function, batched=True, remove_columns=valid_dataset.column_names)
        self.tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=test_dataset.column_names)

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [6]:
import evaluate

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [7]:
# from huggingface_hub import notebook_login

# notebook_login()

In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

dataset = Dataset()

args = Seq2SeqTrainingArguments(
    f"model_small100-finetuned",
    learning_rate=1e-4,
    weight_decay=0.01,
    num_train_epochs=3,

    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="no",
    save_total_limit=0,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=dataset.tokenized_train_dataset,
    eval_dataset=dataset.tokenized_valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

c:\Users\migue\miniconda3\envs\ap\lib\site-packages\accelerate\accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [9]:
# trainer.evaluate(max_length=128)

In [10]:
trainer.train()

  0%|          | 0/1080 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 5}


{'loss': 1.3023, 'grad_norm': 5.048515319824219, 'learning_rate': 5.388888888888889e-05, 'epoch': 1.39}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 5}


{'loss': 0.6783, 'grad_norm': 4.743927955627441, 'learning_rate': 7.592592592592593e-06, 'epoch': 2.78}
{'train_runtime': 328.1085, 'train_samples_per_second': 105.276, 'train_steps_per_second': 3.292, 'train_loss': 0.958622858259413, 'epoch': 3.0}


TrainOutput(global_step=1080, training_loss=0.958622858259413, metrics={'train_runtime': 328.1085, 'train_samples_per_second': 105.276, 'train_steps_per_second': 3.292, 'train_loss': 0.958622858259413, 'epoch': 3.0})

In [11]:
trainer.evaluate(max_length=128)

  0%|          | 0/64 [00:00<?, ?it/s]

{'eval_loss': 1.0073951482772827,
 'eval_bleu': 39.68888408500128,
 'eval_runtime': 86.6174,
 'eval_samples_per_second': 23.471,
 'eval_steps_per_second': 0.739,
 'epoch': 3.0}

In [12]:
import gradio as gr

def translate_text(text):
    tokenizer.tgt_lang = 'pt'
    inputs = tokenizer(text, return_tensors="pt", truncation=True)

    if torch.cuda.is_available():
        inputs = to_device(inputs, 'cuda')

    translated = model.generate(**inputs, max_length=128, num_return_sequences=1)
    decoded_translation = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    return decoded_translation[0]

def chatbot(message):
    translated_message = translate_text(message)
    return translated_message



demo_chatbot = gr.Interface(chatbot, ["textbox"], "text", title="Multilingual Chatbot (small100)", description="Enter text in English and choose a target language for translation.")
demo_chatbot.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.28.3, however version 4.29.0 is available, please upgrade.
--------


In [13]:
# trainer.push_to_hub()